## Import libraries and define parameters

In [21]:
import pandas as pd
import json
import numpy as np
import os
import nlpaug.augmenter.word as naw
import re

from tqdm import tqdm

In [5]:
DATA_PATH = "/Users/Antoine/Desktop/Cours/X-HEC/HEC/Cours/Quinten - drug prescription"


## Import and shape data

In [6]:
with open(os.path.join(DATA_PATH, 'trainset.jsonl'), 'r') as json_file:
    json_list = list(json_file)
ids = np.array([])
texts = np.array([])
labels = []
comments = []
for json_str in tqdm(json_list):
    result = json.loads(json_str)
    ids = np.append(ids, result['id'])
    texts = np.append(texts, result['text'])
    labels.append(result['labels'])
    comments.append(result['Comments'])
df = pd.DataFrame([ids, texts, labels, comments]).T
df.columns = ['id', 'text', 'labels', 'comments']
print(df.shape)
df.head()

100%|██████████| 567/567 [00:12<00:00, 46.72it/s] 


(567, 4)


,id,text,labels,comments
0,1.0,Un homme âgé de 77 ans présentait des signes c...,"[[692, 712, Treatment]]",[]
1,2.0,"Mr. R,R âgé de 53 ans sans antécédents patholo...","[[1224, 1237, Treatment]]",[]
2,3.0,Un homme de 47 ans aux antécédents d’infarctus...,[],[]
3,4.0,"Mme C.S..., âgée de 25 ans, célibataire, sans ...",[],[]
4,5.0,Un adénocarcinome prostatique avait été diagno...,"[[183, 196, Treatment], [1374, 1388, Treatment...",[]


In [15]:
for index, row in tqdm(df.iterrows()):
    text = df.text[index]
    for label in df.labels[index]:
        type = label[-1]
        df.at[index, f"{type}"] = text[label[0]: label[1]]
df.head()

567it [00:00, 2247.01it/s]


,id,text,labels,comments,Treatment,Drug,Duration,Dosage,Route,Frequency,Form
0,1.0,Un homme âgé de 77 ans présentait des signes c...,"[[692, 712, Treatment]]",[],blocage androgénique,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,"Mr. R,R âgé de 53 ans sans antécédents patholo...","[[1224, 1237, Treatment]]",[],radiothérapie,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Un homme de 47 ans aux antécédents d’infarctus...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,"Mme C.S..., âgée de 25 ans, célibataire, sans ...",[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Un adénocarcinome prostatique avait été diagno...,"[[183, 196, Treatment], [1374, 1388, Treatment...",[],chimiothérapie,docétaxel,NaN,NaN,NaN,NaN,NaN


In [3]:
with open(os.path.join(DATA_PATH, 'trainset.jsonl'), 'r') as json_file:
    json_list = list(json_file)

In [4]:
data_list = []
for json_str in tqdm(json_list):
    result = json.loads(json_str)
    data_list.append([result.get('id'), result.get('text')])

df = pd.DataFrame(data_list, columns=[["id", "text"]])
print(df.shape)
df.head(2)

100%|██████████| 567/567 [00:00<00:00, 19284.55it/s]

(567, 2)


,id,text
0,1,Un homme âgé de 77 ans présentait des signes c...
1,2,"Mr. R,R âgé de 53 ans sans antécédents patholo..."


In [16]:
text_sample = df.iloc[0, 1]
text_sample

"Un homme âgé de 77 ans présentait des signes cliniques de prostatisme. Le TR notait une masse rénitente. Le taux de PSA était de 9,5 ng/ml. L'EER et l'imagerie par résonance magnétique nucléaire (IRM) de la prostate montraient dans la ZP droite, une masse kystique à parois épaisses de 4cm de diamètre et contenant des végétations endoluminales. L'analyse histologique des biopsies prostatiques échoguidées montrait un adénocarcinome ductal de score de Gleason 7 (4+3) et de contingent trabéculaire prédominant. L'analyse immuno-histologique pour les cytokératines de haut poids moléculaire (anti CK 903) était négative. Le stade clinique était lors du diagnostic : T3NxM0. Un traitement par blocage androgénique complet a été indiqué. Le PSA à 6 mois était à 0,36 ng/ml puis avait augmenté progressivement jusqu'à atteindre 3,2 ng/ml, 3 ans après le diagnostic. Une EER de contrôle à 3 ans a été réalisée et montrait la persistance de cette masse kystique mais avec des parois et des végétations en

## Apply model
### We use a back translation model taken from nlpaug

In [25]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-fr-en', 
    to_model_name='Helsinki-NLP/opus-mt-en-fr'
)

In [20]:
'blocage androgénique' in text_sample

True

In [7]:
text_transformed = back_translation_aug.augment(text_sample)
text_transformed

["L'examen clinique n'était pas inhabituel. Le toucher rectal était augmenté en volume, non douloureux, d'apparence adénomateuse. Le test PSA s'est avéré à un taux de 30,1 mg/ml (normal < 4 ng/ml). L'échographie endorectale a montré une structure de la prostate homogène. Après un échec d'échantillonnage vésical, une cystostomie minimale a été mise en place. Un ACR a montré une vessie normale et une urètre prostatique dilatée en amont d'une image lacustre à l'urètre prostatique (figure 1). Le patient a reçu une résection transurérale de la prostate avec des séquelles simples. L'examen anatomopathologique de la pièce a montré que tous les copalévaux impliquaient une prolifération tumorale faite de masses et de structures tubulo-papillaires avec une radiothérapie recombinante avec une cellule morphique recombinante et une cellule morphique recombinante."]

Back translation applied on whole sentences change the number of words. It is hence impossible to track the position of labeled words.  
We try to use translation on specific words or expressions.  

In [22]:
def find_position(word, text):
    word_length = len(word)
    first_letter_positions = [m.start() for m in re.finditer(word, text)]
    return [[start, start+word_length] for start in first_letter_positions]

In [42]:
def process(text, label):
    transformed_text, transformed_label = back_translation_aug.augment([text, label])

    if transformed_text==text:
        return 'same back-translation'

    if label in transformed_text:
        new_positions = find_position(label, transformed_text)
        return transformed_text, [new_positions, label]

    if transformed_label in transformed_text:
        new_positions = find_position(transformed_label, transformed_text)
        return transformed_text, [new_positions, transformed_label]

    return 'label and transformed label not in transformed text', transformed_text, transformed_label

In [46]:
result = process(text_sample, 'blocage androgénique')
result

("Un mâle de 77 ans a montré des signes cliniques de prostatisme. TR a noté une masse renitante. PSA était de 9,5 ng/ml. ERA et l'imagerie par résonance magnétique prostatique (NRM) ont montré dans la main droite PZ une masse kystique avec des parois épaisses de 4 cm de diamètre contenant de la végétation endoluminale. L'analyse histologique des biopsies prostatiques échoguides a montré un adénocarcinome canalaire de Gleason 7 score (4+3) et un quota trabéculaire prédominant. L'analyse immuno-histologique des cytokératines de poids moléculaire élevé (anti CK 903) était négative. Le stade clinique était au diagnostic: T3NxM0. Un traitement complet de blocage androgène a été indiqué. Un PSA de 6 mois était de 0,36 ng/ml et a augmenté progressivement à 3,2 ng/ml, 3 ans après le diagnostic.",
 [[[656, 673]], 'blocage androgène'])

In [27]:
transformed_text, new_pos = result

TypeError: cannot unpack non-iterable NoneType object

In [30]:
text_sample

"Un homme âgé de 77 ans présentait des signes cliniques de prostatisme. Le TR notait une masse rénitente. Le taux de PSA était de 9,5 ng/ml. L'EER et l'imagerie par résonance magnétique nucléaire (IRM) de la prostate montraient dans la ZP droite, une masse kystique à parois épaisses de 4cm de diamètre et contenant des végétations endoluminales. L'analyse histologique des biopsies prostatiques échoguidées montrait un adénocarcinome ductal de score de Gleason 7 (4+3) et de contingent trabéculaire prédominant. L'analyse immuno-histologique pour les cytokératines de haut poids moléculaire (anti CK 903) était négative. Le stade clinique était lors du diagnostic : T3NxM0. Un traitement par blocage androgénique complet a été indiqué. Le PSA à 6 mois était à 0,36 ng/ml puis avait augmenté progressivement jusqu'à atteindre 3,2 ng/ml, 3 ans après le diagnostic. Une EER de contrôle à 3 ans a été réalisée et montrait la persistance de cette masse kystique mais avec des parois et des végétations en